In [6]:
import requests
# pprint is used to format the JSON response
from pprint import pprint
import os
import pandas as pd
import numpy as np

##Takes in cpf_tickets.csv and generates cpf_processed.csv
subscription_key = "cd0cf9855b244aa28c017742ed7a904c"
endpoint = "https://cpftext.cognitiveservices.azure.com/"

sentiment_url = endpoint + "/text/analytics/v2.1/sentiment"
language_api_url = endpoint + "/text/analytics/v2.1/languages"
keyphrase_url = endpoint + "/text/analytics/v2.1/keyphrases"
documents = {"documents": []}
training_data = pd.read_csv("cpf_tickets.csv")

#Request for keywords from azure and append
for index, row in training_data.iterrows():
    newobj = {"id": index, "language": "en",
        "text": row['body']}
    documents["documents"].append(newobj)

headers = {"Ocp-Apim-Subscription-Key": subscription_key}
response = requests.post(keyphrase_url, headers=headers, json=documents)
key_phrases = response.json()

keywords = []
for i in range(len(response.json()['documents'])):
    s = ""
    for j in range(len(response.json()['documents'][i]['keyPhrases'])):
        s = s + " " + response.json()['documents'][i]['keyPhrases'][j]
    if s == "":
        s = "NoKeywordsFound"
    keywords.append(s)
training_data['keywords'] = keywords

#Request for sentiment from azure and append
response = requests.post(sentiment_url, headers=headers, json=documents)
sentiments_response = response.json()

sentiments = []
for i in range(len(response.json()['documents'])):
    sentiments.append(response.json()['documents'][i]['score'])
training_data['sentiments'] = sentiments
training_data.to_csv("cpf_processed.csv")